In [25]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn.modules import conv
from torch.nn.modules.utils import _pair
import torch.optim as optim
from torch.autograd import Variable, grad
import torch.nn.functional as F
from torchvision  import transforms, datasets

In [96]:
def l2normalize(v, eps=1e-12):
    return v / (v.norm() + eps)

def max_singular_value(W, u=None, Ip=1):
    """
    power iteration for weight parameter
    """
    if u is None:
        u = torch.FloatTensor(1, W.size(0)).normal_()
        
    _u = u
    for _ in range(Ip):
        _v = l2normalize(torch.matmul(_u, W), eps=1e-12)
        _u = l2normalize(torch.matmul(_v, torch.transpose(W, 0, 1)), eps=1e-12)
        
    sigma = torch.sum(F.linear(_u, torch.transpose(W, 0, 1)) * _v)
    return sigma, _u
    
    
class SNConv2d(conv._ConvNd):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True):
        kernel_size = _pair(kernel_size)
        stride = _pair(stride)
        padding = _pair(padding)
        dilation = _pair(dilation)
        super(SNConv2d, self).__init__(in_channels, out_channels, kernel_size, stride, padding, dilation, False, _pair(0), groups, bias)
        self.u = nn.Parameter(torch.Tensor(1, out_channels).normal_(), requires_grad=False)
        #self.u = torch.Tensor(1, out_channels).normal_()

    @property
    def W_(self):
        w_mat = self.weight.view(self.weight.size(0), -1).data
        sigma, _u = max_singular_value(w_mat, self.u.data)
        self.u.data = _u
        return self.weight / sigma

    def forward(self, input):
        return F.conv2d(input, self.W_, self.bias, self.stride, self.padding, self.dilation, self.groups)

"""class SpectralNorm(nn.Module):
    def __init__(self, module, name='weight', power_iterations=1):
        super(SpectralNorm, self).__init__()
        self.module = module
        self.name = name
        self.power_iterations = power_iterations
        if not self._made_params():
            self._make_params()

    def _update_u_v(self):
        u = getattr(self.module, self.name + "_u")
        v = getattr(self.module, self.name + "_v")
        w = getattr(self.module, self.name + "_bar")

        height = w.data.shape[0]
        for _ in range(self.power_iterations):
            v.data = l2normalize(torch.mv(torch.t(w.view(height,-1).data), u.data))
            u.data = l2normalize(torch.mv(w.view(height,-1).data, v.data))

        # sigma = torch.dot(u.data, torch.mv(w.view(height,-1).data, v.data))
        sigma = u.dot(w.view(height, -1).mv(v))
        setattr(self.module, self.name, w / sigma.expand_as(w))

    def _made_params(self):
        try:
            u = getattr(self.module, self.name + "_u")
            v = getattr(self.module, self.name + "_v")
            w = getattr(self.module, self.name + "_bar")
            return True
        except AttributeError:
            return False


    def _make_params(self):
        w = getattr(self.module, self.name)

        height = w.data.shape[0]
        width = w.view(height, -1).data.shape[1]

        u = nn.Parameter(w.data.new(height).normal_(0, 1), requires_grad=False)
        v = nn.Parameter(w.data.new(width).normal_(0, 1), requires_grad=False)
        u.data = l2normalize(u.data)
        v.data = l2normalize(v.data)
        w_bar = nn.Parameter(w.data)

        del self.module._parameters[self.name]

        self.module.register_parameter(self.name + "_u", u)
        self.module.register_parameter(self.name + "_v", v)
        self.module.register_parameter(self.name + "_bar", w_bar)


    def forward(self, *args):
        self._update_u_v()
        return self.module.forward(*args)


class SNConv2d(SpectralNorm):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True):
        module = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias)
        super(SNConv2d, self).__init__(module)"""

'class SpectralNorm(nn.Module):\n    def __init__(self, module, name=\'weight\', power_iterations=1):\n        super(SpectralNorm, self).__init__()\n        self.module = module\n        self.name = name\n        self.power_iterations = power_iterations\n        if not self._made_params():\n            self._make_params()\n\n    def _update_u_v(self):\n        u = getattr(self.module, self.name + "_u")\n        v = getattr(self.module, self.name + "_v")\n        w = getattr(self.module, self.name + "_bar")\n\n        height = w.data.shape[0]\n        for _ in range(self.power_iterations):\n            v.data = l2normalize(torch.mv(torch.t(w.view(height,-1).data), u.data))\n            u.data = l2normalize(torch.mv(w.view(height,-1).data, v.data))\n\n        # sigma = torch.dot(u.data, torch.mv(w.view(height,-1).data, v.data))\n        sigma = u.dot(w.view(height, -1).mv(v))\n        setattr(self.module, self.name, w / sigma.expand_as(w))\n\n    def _made_params(self):\n        try:\n  

In [99]:
class Generator(nn.Module):
    def __init__(self, zdim=100, n_feature_maps=64):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            #1x1
            nn.ConvTranspose2d(zdim, 8*n_feature_maps, 4, 1, 0, bias=False),
            nn.BatchNorm2d(8*n_feature_maps),
            nn.ReLU(True),
            #4x4
            nn.ConvTranspose2d(8*n_feature_maps, 4*n_feature_maps, 4, 2, 1, bias=False),
            nn.BatchNorm2d(4*n_feature_maps),
            nn.ReLU(True),
            #8x8
            nn.ConvTranspose2d(4*n_feature_maps, 2*n_feature_maps, 4, 2, 1, bias=False),
            nn.BatchNorm2d(2*n_feature_maps),
            nn.ReLU(True),
            #16x16
            nn.ConvTranspose2d(2*n_feature_maps, n_feature_maps, 4, 2, 1, bias=False),
            nn.BatchNorm2d(n_feature_maps),
            nn.ReLU(True),
            #32x32
            nn.ConvTranspose2d(n_feature_maps, 3, 4, 2, 1, bias=False),
            #64x64
            nn.Tanh()
        )
        
    def  forward(self, x):
        return self.main(x)


class SNDiscriminator(nn.Module):
    def __init__(self, n_feature_maps=64):
        super(SNDiscriminator, self).__init__()
        self.main = nn.Sequential(
            #64x64
            #SpectralNorm(nn.Conv2d(3, n_feature_maps, 4, 2, 1, bias=False)),
            SNConv2d(3, n_feature_maps, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            #32x32
            #SpectralNorm(nn.Conv2d(n_feature_maps, 2*n_feature_maps, 4, 2, 1, bias=False)),
            SNConv2d(n_feature_maps, 2*n_feature_maps, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            #16x16
            #SpectralNorm(nn.Conv2d(2*n_feature_maps, 4*n_feature_maps, 4, 2, 1, bias=False)),
            SNConv2d(2*n_feature_maps, 4*n_feature_maps, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            #8x8
            #SpectralNorm(nn.Conv2d(4*n_feature_maps, 8*n_feature_maps, 4, 2, 1, bias=False)),
            nn.LeakyReLU(0.2, inplace=True)
            )
        self.output = nn.Sequential(
            # 4x4
            #SpectralNorm(nn.Conv2d(8*n_feature_maps, 1, 4, 1, 0, bias=False)),
            SNConv2d(4*n_feature_maps, 1, 4, 1, 0, bias=False),
            #1x1
            nn.Sigmoid()
        )
        
    def  forward(self, x, matching=False):
        x = self.main(x)
        if matching :
            return x
        else:
            output = self.output(x)
            return output.view(-1, 1).squeeze(1)

In [100]:
D = SNDiscriminator(64)
t = Variable(torch.rand(20,3,64,64))
D(t)

Variable containing:
 0.5118
 0.5235
 0.5206
 0.5283
 0.4850
 0.5419
 0.5254
 0.5065
 0.5349
 0.5075
 0.5220
 0.5577
 0.5152
 0.5197
 0.5225
 0.5079
 0.5287
 0.4900
 0.4662
 0.4714
 0.5294
 0.5244
 0.5328
 0.5440
 0.4695
 0.5066
 0.5182
 0.5350
 0.5342
 0.4870
 0.5090
 0.5094
 0.5284
 0.4809
 0.5127
 0.5358
 0.4933
 0.5171
 0.4968
 0.5151
 0.4966
 0.4908
 0.5026
 0.4967
 0.5212
 0.4942
 0.5043
 0.5220
 0.5163
 0.4821
 0.4984
 0.5195
 0.5125
 0.5447
 0.4877
 0.5121
 0.5081
 0.5106
 0.4983
 0.5039
 0.4664
 0.5190
 0.5098
 0.5310
 0.4957
 0.4943
 0.5067
 0.4898
 0.4827
 0.5078
 0.5510
 0.5058
 0.5580
 0.5208
 0.5280
 0.5268
 0.5063
 0.5398
 0.5408
 0.4798
 0.5112
 0.4952
 0.5269
 0.5426
 0.4855
 0.4930
 0.5483
 0.5172
 0.5268
 0.4823
 0.4901
 0.5149
 0.5021
 0.4715
 0.4949
 0.5488
 0.5213
 0.5153
 0.5243
 0.4813
 0.5305
 0.5017
 0.5077
 0.5175
 0.5428
 0.4984
 0.5136
 0.5430
 0.5166
 0.4931
 0.5554
 0.5250
 0.4942
 0.5443
 0.5336
 0.4887
 0.4896
 0.5344
 0.5009
 0.4919
 0.4976
 0.5265
 0.

In [101]:
batch_size = 64

transform = transforms.Compose(
	[
	    transforms.ToTensor(),
	    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
	])
dataset = datasets.ImageFolder('paintings64/', transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

In [102]:
zdim = 100
n_feature_maps = 128
n_epochs = 50

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv' or 'SNConv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

G = Generator(zdim, n_feature_maps)
G.apply(weights_init)
D = SNDiscriminator(n_feature_maps)
D.apply(weights_init)

SNDiscriminator(
  (main): Sequential(
    (0): SNConv2d (3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(0.2, inplace)
    (2): SNConv2d (128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): LeakyReLU(0.2, inplace)
    (4): SNConv2d (256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (5): LeakyReLU(0.2, inplace)
    (6): LeakyReLU(0.2, inplace)
  )
  (output): Sequential(
    (0): SNConv2d (512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): Sigmoid()
  )
)

In [56]:
lr = 1e-3
beta1 = 0.5
beta2 = 0.999
G_optimiser = optim.Adam(G.parameters(), lr=lr, betas=(beta1, beta2))
D_optimiser = optim.Adam(D.parameters(), lr=lr, betas=(beta1, beta2))

In [103]:
for epoch in range(1,n_epochs+1):
    D_losses = []
    G_losses = []
    for img, label in dataloader:
        print('kjd')
        x = Variable(img)

        # D training, n_critic=1
        for p in D.parameters():
            p.requires_grad = True

        D.zero_grad
        D_real = D(x)

        z = torch.FloatTensor(x.size(0), zdim, 1, 1).normal_()

        z = Variable(z)
        fake = G(z)
        D_fake = D(fake.detach())        
        D_err = torch.mean(D_real) - torch.mean(D_fake)
        D_optimiser.step()

        # G training
        for p in D.parameters():
            p.requires_grad = False # saves computation

        z = torch.FloatTensor(batch_size, zdim, 1, 1).normal_()

        z = Variable(z)
        fake = G(z)
        G_err = torch.mean(D(fake))
        G_optimiser.step()

kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd
kjd


Process Process-17:
Process Process-18:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/francois/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/francois/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/francois/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/francois/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/francois/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/home/francois/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/francois/anaconda3/envs/pytorch/l

KeyboardInterrupt: 